# Extract PRISM data
- using 10km scale and WGS84 projection
- For entire CONUS
- images with data for 3 months
- Note: When converting an image collection to an image, GEE includes the date in the band name. Timestamps on PRISM data are midday, so when they are converted to a date, this becomes the following day. So band names for 2016-10-01 data will include 2016-10-02!

In [1]:
import initialise
import common
import ee
ee.Initialize()

In [2]:
PRODUCT = "OREGONSTATE/PRISM/AN81d"
PROJ = common.MAP_PROJ
SCALE = 4638.3       # PRISM native resolution      # common.MAP_SCALE
USA = ee.FeatureCollection('USDOS/LSIB/2017').filter("COUNTRY_NA == 'United States'")
FLOAT_PRE = 5
START_DATES = ["2017-10-01", "2018-01-01", "2018-04-01", "2018-07-01"]
END_DATES = ["2018-01-01", "2018-04-01", "2018-07-01", "2018-10-01"]
bbox = ee.Geometry.BBox(*common.MAP_BOUNDS)
conus_bbox = bbox.intersection(USA.geometry(), 1, PROJ)
FOLDER = f"GEE_{PROJ.replace(':', '-')}_{int(SCALE)}"

In [3]:
images = []
for num in range(len(START_DATES)):
    print(START_DATES[num], END_DATES[num])
    prism_col = ee.ImageCollection(PRODUCT).filterDate(START_DATES[num], END_DATES[num])
    images.append(prism_col.toBands().clip(conus_bbox).regexpRename('^', 'b'))

2017-10-01 2018-01-01
2018-01-01 2018-04-01
2018-04-01 2018-07-01
2018-07-01 2018-10-01


In [4]:
for num, image in enumerate(images):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'PRISM_{START_DATES[num]}',
        folder='CONUS',
        fileFormat='GeoTIFF',
        scale=SCALE,
        crs=PROJ,
        skipEmptyTiles=True
    );
    task.start()